# Challenge 1

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
def is_running_on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ and os.environ["KAGGLE_KERNEL_RUN_TYPE"] == "Interactive"
data_path = '/kaggle/input/aerial-cactus/' if is_running_on_kaggle() else 'data/'
print('Running on Kaggle' if is_running_on_kaggle() else 'Running locally')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Running locally


In [4]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Set device to CUDA
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    # Set device to CPU
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

torch.manual_seed(42)

CUDA is available. Using GPU.


In [5]:
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

# Imports
import warnings
import matplotlib.pyplot as plt
from torchvision.io import read_image

class CactusDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
annotations_file = data_path + 'train.csv'
img_dir = data_path + 'train/train/'

BATCH_SIZE = 64
LEARNING_RATE = 1e-3

TODO: Try to preprocess like in ImProc

In [6]:
from torchvision.models import resnet18
from torchvision.transforms import Normalize
import torchvision.transforms as transforms

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ConvertImageDtype(torch.float32),
    transforms.Normalize(mean=mean, std=std),
])
training_dataset = CactusDataset(annotations_file, img_dir, transform=transform)

print(training_dataset[0][0].shape)

torch.Size([3, 224, 224])


In [7]:
from torch.utils.data import random_split

# Define the sizes of training and validation sets
train_size = int(0.6 * len(training_dataset))
val_size = len(training_dataset) - train_size

# Split the dataset into training and validation sets
train_data, val_data = random_split(training_dataset, [train_size, val_size])

# Print the sizes of the training and validation sets
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))

Training set size: 10500
Validation set size: 7000


In [8]:
import torch.nn as nn

class ResnetClassificator(nn.Module):
    def __init__(self):
        super(ResnetClassificator, self).__init__()
        self.resnet = resnet18(pretrained=True)
        last_layer_size = self.resnet.fc.out_features # 1000        
        self.fc1 = nn.Linear(last_layer_size, 8)
        self.fc2 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [9]:
model = ResnetClassificator()
model.to(device)

/home/andrewl73/anaconda3/envs/AML/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/andrewl73/anaconda3/envs/AML/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResnetClassificator(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [10]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
train_dataloader = torch.utils.data.DataLoader(train_data, BATCH_SIZE, shuffle=True)

do_train = True

def train_model(train_dataloader, model, criterion, optimizer):
    for batch, (X, y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X.float())
        # print(pred)
        # print('Interval: ', pred.min(), pred.max())
        y = y.reshape(-1, 1).float()
        loss = criterion(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"batch {batch} | loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

if do_train:
    epochs = 5
    # for t in range(epochs):
    #     print(f"Epoch {t+1}\n-------------------------------")
    #     train_model(train_dataloader, model, criterion, optimizer)
    train_model(train_dataloader, model, criterion, optimizer)

batch 0 | loss: 0.730981  [    0/10500]
batch 10 | loss: 0.131074  [  640/10500]


KeyboardInterrupt: 

In [ ]:
val_dataloader = torch.utils.data.DataLoader(val_data, BATCH_SIZE)
model.eval()

errors = []

with torch.no_grad():
    for batch, (X, y) in enumerate(val_dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X.float())
        y = y.reshape(-1, 1).float()
        loss = criterion(pred, y)

        pred = nn.Sigmoid()(pred) > 0.5
        errors += pred != y
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{val_size:>5d}]")

loss: 0.135296  [    0/ 7000]
loss: 0.207270  [  640/ 7000]
loss: 0.045310  [ 1280/ 7000]
loss: 0.151156  [ 1920/ 7000]
loss: 0.201006  [ 2560/ 7000]
loss: 0.139554  [ 3200/ 7000]
loss: 0.243063  [ 3840/ 7000]
loss: 0.247136  [ 4480/ 7000]
loss: 0.278913  [ 5120/ 7000]
loss: 0.092146  [ 5760/ 7000]
loss: 0.104191  [ 6400/ 7000]


In [ ]:
# print(matches)
errors = torch.tensor(errors)
print(f'Error rate: {errors.sum().item() / val_size}')
print(f'Number of errors: {errors.sum().item()} over {len(errors)} samples')

Error rate: 0.04542857142857143
Number of errors: 318 over 7000 samples
